In [157]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
matplotlib.style.use('fivethirtyeight')
import json

test_data = {}

files = ["TEST DATA/" + i for i in ['my_dict (2).json', 'my_dict (4).json', 'my_dict.json']]

for name in files:
    with open(name, 'r') as f:
        test_data.update(json.load(f))

In [158]:
test_data = {k: test_data[k] for k in list(test_data.keys())[:50]}

In [218]:
from scipy import stats
summary = {"UDS": [],
           "MRI": [],
           "CSF":[],
           "Simple Fusion": [],
           "Early Fusion": [],
           "Late Fusion": []}

multiclass_vals = []
binary_vals = []

def big_data(name, coords): 
    all_uds = []
    all_uds_ensemble = []
    for key in test_data:
        all_uds.append(test_data[key][name][0:4])
        
        
        all_uds_ensemble.append(test_data[key][name][4:])
        

    # SINGLE MODELS
    mean_values = np.mean(np.stack(all_uds), axis=0)*100
    conf_interval_size = stats.sem(np.stack(all_uds), axis=0) * stats.t.ppf((1 + 0.95) / 2, 2)*100

    format_func = np.vectorize(lambda mean, conf: f"{mean:.2f} ± {conf:.2e}")
    formatted_array = format_func(mean_values, conf_interval_size)
    
    for i in range(len(mean_values)):
            for j in range(1, len(mean_values[i])):
                multiclass_vals.append([[mean_values[i][j]*100, conf_interval_size[i][j]*100], name, False])
            binary_vals.append([[mean_values[i][0]*100, conf_interval_size[i][0]*100], name, False])
                
    
    # PASTE TO EXCEL
    start_row = coords[0]; start_col = coords[1]
    wb = load_workbook('Book1.xlsx'); ws = wb.active; [[ws.cell(row=start_row + i, column=start_col + j, value=formatted_array[i, j]) for j in range(formatted_array.shape[1])] for i in range(formatted_array.shape[0])]; wb.save('Book1.xlsx')

    # ENSEMBLE MODELS
    mean_values = np.mean(np.stack(all_uds_ensemble), axis=0)
    conf_interval_size = stats.sem(np.stack(all_uds_ensemble), axis=0) * stats.t.ppf((1 + 0.95) / 2, 2)
    
    multiclass_vals.append([[mean_values[0]/100,conf_interval_size[0]/100], name, True])
    binary_vals.append([[mean_values[1]/100, conf_interval_size[1]/100], name, True])

    format_func = np.vectorize(lambda mean, conf: f"{mean:.2f} ± {conf:.2e}")
    formatted_array = format_func(mean_values, conf_interval_size)
    formatted_array = np.array([[x] for x in formatted_array])

    # PASTE TO EXCEL
    start_row = coords[2]; start_col = coords[3]
    wb = load_workbook('Book1.xlsx'); ws = wb.active; [[ws.cell(row=start_row + i, column=start_col + j, value=formatted_array[i, j]) for j in range(formatted_array.shape[1])] for i in range(formatted_array.shape[0])]; wb.save('Book1.xlsx')

big_data("UDS", [4, 3, 5, 1])
big_data("MRI", [12, 3, 13, 1])
big_data("Late Fusion", [29, 7, 26, 6])
big_data("Early Fusion", [29, 2, 26, 1])

In [219]:
from scipy import stats
summary = {"UDS": [],
           "MRI": [],
           "CSF":[],
           "Simple Fusion": [],
           "Early Fusion": [],
           "Late Fusion": []}


def smol_data(name, coords): 
    all_uds = []
    all_uds_ensemble = []
    for key in test_data:
        all_uds.append(test_data[key][name][0])
        
        
        all_uds_ensemble.append(test_data[key][name][1:])
        

    # SINGLE MODELS
    mean_values = np.mean(np.stack(all_uds), axis=0)*100
    conf_interval_size = stats.sem(np.stack(all_uds), axis=0) * stats.t.ppf((1 + 0.95) / 2, 2)*100

    format_func = np.vectorize(lambda mean, conf: f"{mean:.2f} ± {conf:.2e}")
    formatted_array = format_func(mean_values, conf_interval_size)
    formatted_array = np.array([formatted_array])
    
    for i in range(1,len(mean_values)):
        multiclass_vals.append([[mean_values[i]*100, conf_interval_size[i]*100], name, False])
    binary_vals.append([[mean_values[0]*100, conf_interval_size[0]*100],name, False])
    
    # PASTE TO EXCEL
    start_row = coords[0]; start_col = coords[1]
    wb = load_workbook('Book1.xlsx'); ws = wb.active; [[ws.cell(row=start_row + i, column=start_col + j, value=formatted_array[i, j]) for j in range(formatted_array.shape[1])] for i in range(formatted_array.shape[0])]; wb.save('Book1.xlsx')

    # ENSEMBLE MODELS
    mean_values = np.mean(np.stack(all_uds_ensemble), axis=0)
    conf_interval_size = stats.sem(np.stack(all_uds_ensemble), axis=0) * stats.t.ppf((1 + 0.95) / 2, 2)

    format_func = np.vectorize(lambda mean, conf: f"{mean:.2f} ± {conf:.2e}")
    formatted_array = format_func(mean_values, conf_interval_size)
    formatted_array = np.array([[x] for x in formatted_array])

    multiclass_vals.append([[mean_values[0], conf_interval_size[0]], name, True])
    # PASTE TO EXCEL
    start_row = coords[2]; start_col = coords[3]
    wb = load_workbook('Book1.xlsx'); ws = wb.active; [[ws.cell(row=start_row + i, column=start_col + j, value=formatted_array[i, j]) for j in range(formatted_array.shape[1])] for i in range(formatted_array.shape[0])]; wb.save('Book1.xlsx')

smol_data("Simple Fusion", [5, 8, 6, 7])
smol_data("CSF", [20, 2, 21, 1])



In [204]:
# multiclass_vals = [[[x[0][0]/100, x[0][1]], x[1], x[2]]if x[0][0] > 50 else x for x in multiclass_vals  ]
# binary_vals = [[[x[0][0]/100, x[0][1]], x[1], x[2]] if x[0][0] > 50 else x for x in binary_vals  ]

In [205]:
sorted_multiclass = list(reversed(sorted(multiclass_vals, key=lambda x: x[0][0])))
sorted_multiclass

[[[86.7002809047699, 0.9777598042869484], 'Late Fusion', False],
 [[86.43697478991598, 0.757640340372454], 'Late Fusion', False],
 [[83.3501400560224, 1.1879624403370175], 'Early Fusion', False],
 [[83.10364145658264, 1.2201072723109798], 'Simple Fusion', False],
 [[81.85994397759104, 1.2240466493256856], 'Late Fusion', False],
 [[81.84313833713531, 1.2371175888239045], 'Early Fusion', False],
 [[81.57422969187675, 1.1186286707115694], 'Simple Fusion', True],
 [[80.96780850185719, 0.3150145027909137], 'UDS', False],
 [[80.93190240859985, 0.3443683656506137], 'UDS', False],
 [[80.85154175758362, 1.1832617481881067], 'Late Fusion', False],
 [[80.63516294956207, 0.29280005277783955], 'UDS', False],
 [[80.35534461411474, 0.29791367271281655], 'UDS', False],
 [[80.25210084033615, 1.186509238076519], 'Late Fusion', False],
 [[79.96638774871826, 1.0846968383460953], 'Late Fusion', False],
 [[79.48459506034851, 1.1911521782364751], 'Simple Fusion', False],
 [[79.35014005602244, 1.3859989577427

In [206]:
sorted_binary = list(reversed(sorted(binary_vals, key=lambda x: x[0][0])))
sorted_binary

[[[91.5910364145658, 0.8042543740469154], 'Late Fusion', False],
 [[88.03590590177465, 0.273824212357892], 'UDS', False],
 [[87.74789915966383, 1.0471919460460384], 'Late Fusion', False],
 [[87.63025210084034, 1.0280792775249905], 'Early Fusion', False],
 [[87.109243697479, 0.8956343166135268], 'Late Fusion', False],
 [[86.28011204481793, 1.0946267192520134], 'Early Fusion', False],
 [[86.2352455633512, 0.2642402123443901], 'UDS', False],
 [[85.02521008403357, 0.9544210580870855], 'Early Fusion', False],
 [[84.25423029302519, 0.3027824518327855], 'UDS', False],
 [[80.76479606823835, 1.1735686810404935], 'CSF', False],
 [[79.90476190476191, 1.7029872363217144], 'Simple Fusion', False],
 [[79.84873949579833, 1.3195434690618675], 'Early Fusion', False],
 [[79.25603864734299, 0.9491106660913552], 'MRI', False],
 [[78.53140096618357, 1.157546025379183], 'MRI', False],
 [[78.08212560386472, 1.1353071528169354], 'MRI', False],
 [[75.96078431372551, 1.9831358955184675], 'Late Fusion', False],
